In [2]:
import torch
import torch.nn as nn

#### Wrapper to get a name for convolution layer.

In [47]:
class Conv1d(nn.Module):
    """Module allows us to save the name of our layers for cache."""
    def __init__(self, out_channels, kernel_size):
        super(Conv1d, self).__init__()
        
        self.conv = nn.Sequential()
        self.conv.add_module(
            f"conv1d_{str(kernel_size)}_{str(out_channels)}",
            nn.Conv1d(1, out_channels, kernel_size)
        )
    
    def forward(self, x):
        return self.conv(x)

In [111]:
class Embedding(nn.Module):
    """Wrapper for embedding to save names in cache."""
    def __init__(self, vocab_size, word_dim):
        super(Embedding, self).__init__()
        
        self.emb = nn.Sequential()
        self.emb.add_module(
            f"embedding_{str(vocab_size)}_{str(word_dim)}",
            nn.Embedding(vocab_size + 2, word_dim, padding_idx=0)
        )
        
    def forward(self, x):
        return self.embedding(x)

#### Compose multiple convolution layers together

In [112]:
class CNN(nn.Module):
    
    def __init__(self, hparams):
        super(CNN, self).__init__()
        self.hp = hparams
        self.embedding = Embedding(hparams.vocab_size, hparams.word_dim)
        self.conv1 = Conv1d(hparams.n_filters1, hparams.kernel1)
        self.conv2 = Conv1d(hparams.n_filters2, hparams.kernel2)
        self.conv3 = Conv1d(hparams.n_filters3, hparams.kernel3)
        self.fc = nn.Linear(self._sum_filters(), 10)
        
    def _sum_filters(self):
        return self.hp.n_filters1 + self.hp.n_filters2 + self.hp.n_filters3
        
    def forward(self, x):
        x = self.embedding(x).view(-1, 1, self.word_dim * self.max_sent_len)
        
        conv_results = []
        conv_results.append(nn.ReLU(self.conv1(x)).view(-1, self.hp.n_filters1))
        conv_results.append(nn.ReLU(self.conv2(x)).view(-1, self.hp.n_filters2))
        conv_results.append(nn.ReLU(self.conv2(x)).view(-1, self.hp.n_filters3))
        x = torch.cat(conv_results, 1)
        
        x = self.fc(x)
        return x

In [113]:
class Hyperparameters:
    kernel1 = 3
    kernel2 = 4
    kernel3 = 5
    n_filters1 = 300
    n_filters2 = 300
    n_filters3 = 300
    vocab_size = 3000
    word_dim = 100

In [114]:
hparams = Hyperparameters()
hparams.n_filters1 = 4
model = CNN(hparams)

In [115]:
torch.save(model.state_dict(), 'model')

In [116]:
save = torch.load('model')
model = CNN(hparams)
model.load_state_dict(save, strict=False)

In [117]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

embedding.emb.embedding_3000_100.weight
conv1.conv.conv1d_3_4.weight
conv1.conv.conv1d_3_4.bias
conv2.conv.conv1d_4_300.weight
conv2.conv.conv1d_4_300.bias
conv3.conv.conv1d_5_300.weight
conv3.conv.conv1d_5_300.bias
fc.weight
fc.bias


In [118]:
def new(hyperparameters=None, savefile=None):
    """Create new MTCNN model."""
    if hyperparameters:
        hparams = hyperparameters
    else:
        hparams = Hyperparameters()
        
    model = CNN(hparams)
        
    if savefile:
        model.load_state_dict(savefile, strict=False)
        
    return model

In [119]:
hyperspace_update = {
    'kernel1': 1,
    'kernel2': 2
}

In [120]:
hparams.kernel1 = hyperspace_update['kernel1']

In [121]:
new()

CNN(
  (embedding): Embedding(
    (emb): Sequential(
      (embedding_3000_100): Embedding(3002, 100, padding_idx=0)
    )
  )
  (conv1): Conv1d(
    (conv): Sequential(
      (conv1d_3_300): Conv1d(1, 300, kernel_size=(3,), stride=(1,))
    )
  )
  (conv2): Conv1d(
    (conv): Sequential(
      (conv1d_4_300): Conv1d(1, 300, kernel_size=(4,), stride=(1,))
    )
  )
  (conv3): Conv1d(
    (conv): Sequential(
      (conv1d_5_300): Conv1d(1, 300, kernel_size=(5,), stride=(1,))
    )
  )
  (fc): Linear(in_features=900, out_features=10, bias=True)
)